# INFO

This notebook will be used for construction and testing purposes while designing model within Kedro framework. 

In [1]:
##############################################################################
# It is recommended to create new virtual environment for each Kedro project #
##############################################################################

# Uncomment and run the line below if your environment does't have
# Kedro or any other dependencies needed.

#! pip install -r requirements.txt
%load_ext kedro.ipython

[11/07/24 22:08:12] INFO     Using                                                                  ]8;id=23650;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=393320;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/k                
                             edro/framework/project/rich_logging.yml' as logging configuration.                    

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=927726;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=660864;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#61\61]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=603614;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=935418;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#63\63]8;;\

                    INFO     Resolved project path as:                                              ]8;id=770078;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=112661;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#178\178]8;;\
                             /Users/andrejcemeris/Projects/Python3/lending-club.                                   
                             To set a different path, run '%reload_kedro <project_root>'                           

[11/07/24 22:08:14] INFO     Kedro project lending_club                                             ]8;id=573114;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=381729;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#144\144]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=853525;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=603780;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#145\145]8;;\
                             'pipelines'                                                                           

[11/07/24 22:08:16] INFO     Registered line magic 'run_viz'                                        ]8;id=4701;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=740666;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#151\151]8;;\

# Model
As the dataset needs transformation like imputation and normalization, for avoiding data leakage, all transformations will be done within model pipeline and fitting only on training data on model fitting stage. So I'm going to split initial typed dataset to train/test sets and balance train set, ignoring all previously transformed datasets.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import pandas as pd
import logging
from lending_club.pipelines.analysis.nodes import features_eng
from lending_club.pipelines.encode.nodes import _default_status
from imblearn.pipeline import make_pipeline as imb_make_pipeline

logger = logging.getLogger(__name__)

def split_dataset(df: pd.DataFrame, params: dict):
    y = _default_status(df, params)
    X = df
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=params['test_size'],
        random_state=params['random_state']
    )
    return X_train, X_test, y_train, y_test

def train_model(X_train, y_train, regressor, params: dict):
    try:
        regressor.set_params(**params['fit_options']).fit(X_train, y_train)
    except:
        regressor.fit(X_train, y_train)
    return regressor


def model_pipeline(model_options: dict, params: dict):

    # split important features to assign preprocessing steps
    category_feat = [f for f in (params['category'] + [params['emp_len']]) if f in params['model_features']]
    numeric_feat_zero = [f for f in params['fill_zero'] if f in params['model_features']]
    numeric_feat_med = [f for f in params['fill_med'] if f in params['model_features']]

    # transformer to replace missing numeric values by 0
    # and standartize all values 
    numeric_feat_zero_transformer = make_pipeline(
        SimpleImputer(strategy='constant', fill_value=0),
        StandardScaler()
    )
    # transformer to replace missing numeric values by median
    numeric_feat_med_transformer = make_pipeline(
        SimpleImputer(strategy='median'),
        StandardScaler()
    )

    # assemble transformers in preprocessing pipe so it will perform 
    # following transformations:
    #   - encode all categorical features to numbers
    #   - fill missing values in specific number features as "0" and standartize them
    #   - fill missing values in specific number features as median and standartize them
    preprocessing = make_column_transformer(
        (OrdinalEncoder(), category_feat),
        (numeric_feat_zero_transformer, numeric_feat_zero),
        (numeric_feat_med_transformer, numeric_feat_med)
    )

    # choose regressor depending on provided model_options
    if model_options['name'] == 'rfc':
        regressor = RandomForestClassifier(**model_options['regressor_options'])
    else: 
        if model_options['name'] == 'catboost':
            regressor = CatBoostClassifier(cat_features=category_feat, **model_options['regressor_options'])
        else:
            raise Exception("Pipeline accepts only RandomForestClassifier and CatBoostClassifier")
    
    # assebmle preprocessing pipeline, SMOTE (ibmalance handler) and 
    # choosen regressor as the model pipeline 
    model = imb_make_pipeline(
        preprocessing,
        SMOTE(random_state=params['random_state']),
        regressor
    )
    return model


In [61]:
# Load general parameters
params = catalog.load("parameters")
params['model_features'] = catalog.load("params:model_features")

[11/07/24 10:02:28] INFO     Loading data from parameters (MemoryDataset)...                    data_catalog.py:389

[11/07/24 10:02:29] INFO     Loading data from params:model_features (MemoryDataset)...         data_catalog.py:389

                    INFO     Loading data from params:baseline_model.model_options              data_catalog.py:389
                             (MemoryDataset)...                                                                    

In [62]:
# Load intermediate dataset that has proper features types
# and split it to training/testing features and target datasets
df = catalog.load("intermediate_lc_dataset")
features_eng(df, params)
X_train, X_test, y_train, y_test = split_dataset(df, params)

                    INFO     Loading data from intermediate_lc_dataset (ParquetDataset)...      ]8;id=545689;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=808126;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

In [69]:
# Define model pipeline that will transform original features datasets,
# applying isolated transformations for training and testing datasets
# to prevent data leakage 
def model_pipeline(model_options: dict, params: dict):

    # split important features to assign preprocessing steps
    category_feat = [f for f in (params['category'] + [params['emp_len']]) if f in params['model_features']]
    numeric_feat_zero = [f for f in params['fill_zero'] if f in params['model_features']]
    numeric_feat_med = [f for f in params['fill_med'] if f in params['model_features']]

    # transformer to replace missing numeric values by 0
    # and standardize all values 
    numeric_feat_zero_transformer = make_pipeline(
        SimpleImputer(strategy='constant', fill_value=0),
        StandardScaler()
    )
    # transformer to replace missing numeric values by median
    numeric_feat_med_transformer = make_pipeline(
        SimpleImputer(strategy='median'),
        StandardScaler()
    )

    # assemble transformers in preprocessing pipe so it will perform 
    # following transformations:
    #   - encode all categorical features to numbers
    #   - fill missing values in specific number features as "0" and standardize them
    #   - fill missing values in specific number features as median and standardize them
    preprocessing = make_column_transformer(
        (OrdinalEncoder(), category_feat),
        (numeric_feat_zero_transformer, numeric_feat_zero),
        (numeric_feat_med_transformer, numeric_feat_med)
    )

    # choose regressor depending on provided model_options
    if model_options['name'] == 'rfc':
        regressor = RandomForestClassifier(**model_options['regressor_options'])
    else: 
        if model_options['name'] == 'catboost':
            regressor = CatBoostClassifier(cat_features=category_feat, **model_options['regressor_options'])
        else:
            raise Exception("Pipeline accepts only RandomForestClassifier and CatBoostClassifier")
    
    # assemble preprocessing pipeline, SMOTE (imbalance handler) and 
    # chosen regressor as the model pipeline 
    model = imb_make_pipeline(
        preprocessing,
        SMOTE(random_state=params['random_state']),
        regressor
    )
    return model

# Function that fits model but set parameters for regressor first if it is available 
def train_model(X_train, y_train, regressor, params: dict):
    try: 
        regressor.set_params(**params['fit_options']).fit(X_train, y_train)
    except:
        regressor.fit(X_train, y_train)
    return regressor

# Load base model's options
params['model_options'] = catalog.load("params:baseline_model.model_options")

# Make a model and fit it
model = model_pipeline(params['model_options'], params)
train_model(X_train, y_train, model, params['model_options'])
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['home_ownership',
                                                   'sub_grade',
                                                   'verification_status']),
                                                 ('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['mths_since_rcnt_il']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('si...
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['open_rv_24m', 'open_acc_6m',
                                                   'max_bal_bc',
                                                   'mths_since_recent_inq',
                                                   'percent_bc_gt_75',
                                                   'bc_open_to_buy',
                                                   'tot_cur_bal',
                                                   'mo_sin_rcnt_rev_tl_op',
                                                   'acc_open_past_24mths'])])),
                ('smote', SMOTE(random_state=42)),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='log_loss',
                                        random_state=42))])

# Evaluation
## Actual profit/loss

To evaluate model performance I want to use custom loss function, so I need to calculate actual earning rate to define potential losses in case we refuse in loan, that was mistakenly predicted as default, as well as actual losses for charged off loans, that will be our loss in case if we issue a loan that was mistakenly predicted as non default. 

I assume, that earning rate for non defaulted loans, considering loans that is not fully paid at the moment, is total received amount less than total received principal divided by total received principal. 

For charged off loan, I believe, the actual losses are amount of loan less than total received payments (that includes collections after charges off) plus collection recovery fee (that I believe is our payment to collectors for collection services). Dividing that by this category loan amount we can get actual loss rate for defaulted loans

In [6]:
# Function that returns actual profit/loss rates for non-defartet/defaulted loans
def get_loss_values(df: pd.DataFrame) -> pd.DataFrame:

    # Select columns for profit/loss calculation
    df = df.loc[:, ['loan_amnt', 'loan_status', 'total_pymnt', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee']]

    # Add default status and summarize data
    df['default_status'] = df['loan_status'].str.contains("Charged Off", regex=False, na=False)
    df = df.drop(columns=['loan_status'])
    df=df.groupby(by='default_status').sum()
    df = df.reset_index()


    df['earning/loss'] = (
        # actual earnings rate for non-defaulters
        ((df.total_pymnt - df.total_rec_prncp) / df.total_rec_prncp) * ~df.default_status
        # actual losses rate for defaulters
        + (df.loan_amnt - df.total_pymnt + df.collection_recovery_fee) / df.loan_amnt * df.default_status
        )

    # Select columns: 'default_status', 'earning/loss'
    df = df.loc[:, ['default_status', 'earning/loss']]
    return df.set_index('default_status')

df_loss = get_loss_values(catalog.load('intermediate_lc_clean'))
df_loss

[11/07/24 22:09:35] INFO     Loading data from intermediate_lc_clean (ParquetDataset)...        ]8;id=362177;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=852286;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

,earning/loss
default_status,
False,0.228696
True,0.460871


>These figures will be used in parameters...yml to feed FP_cost and FN_cost to model evaluator

In [7]:
params['FP_cost'] = df_loss['earning/loss'].loc[False]
params['FN_cost'] = df_loss['earning/loss'].loc[True]

## Baseline model
To evaluate my model I'll use a range of probability thresholds that will be used for classification of predicted default's probabilities and calculate metrics for each of them to decide which one is fitting best to minimize losses

In [8]:
def make_rng(start, stop, step):
    return range(start, stop, step)


def evaluate_metrics(model: object, X_true, y_true,
                     params: dict) -> pd.DataFrame:
    y_pred_proba = model.predict_proba(X_true)
    metrics = pd.DataFrame()
    for thresh in make_rng(**params['model_options']['prob_threshold']):
        y_pred = (y_pred_proba[:,1] > (thresh / 100))
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        cur_metrics = pd.DataFrame(
        data={
            'prob_thresh_%': thresh,
            'accuracy'     : accuracy_score(y_true, y_pred),
            'precision'    : precision_score(y_true, y_pred),
            'recall'       : recall_score(y_true, y_pred),
            'f1'           : f1_score(y_true, y_pred),
            'roc_auc'      : roc_auc_score(y_true, y_pred),
            'tn'           : tn,
            'fp'           : fp,
            'fn'           : fn,
            'tp'           : tp,
            'loss'         : params['FP_cost'] * fp + params['FN_cost'] *fn,
        },
        index = [params['model_options']['name']]
        )
        metrics = pd.concat([metrics, cur_metrics], axis=0)
        logger.info(f"The best probability threshold for {params['model_options']['name']} model based on min loss: {eval_metr[eval_metr.loss==eval_metr.loss.min()]['prob_thresh_%'].iloc[0]}")
    return metrics

eval_metr = evaluate_metrics(model, X_test, y_test, params)

eval_metr

The best probability threshold for rfc model based on min loss: 53


The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\chemerys\AppData\Local\Temp\ipykernel_24248\1912064199.py:1 in <module>                 │
│                                                                                                  │
│ ❱ 1 eval_metr = evaluate_metrics(model, X_test, y_test, params)                                  │
│   2                                                                                              │
│   3 eval_metr                                                                                    │
│   4                                                                                              │
│                                                                                                  │
│ in evaluate_metrics:67                                                                           │
│                                                                                                  │
│    64 │   │   index = [params['model_options']['name']]                                          │
│    65 │   │   )                                                                                  │
│    66 │   │   metrics = pd.concat([metrics, cur_metrics], axis=0)                                │
│ ❱  67 │   │   logger.info(f"The best probability threshold for {params['model_options']['name'   │
│    68 │   return metrics                                                                         │
│    69                                                                                            │
│    70 def model_pipeline(model_options: dict, params: dict):                                     │
│                                                                                                  │
│ d:\Program Files\Python\Python Venv\lc\lib\site-packages\pandas\core\frame.py:4102 in            │
│ __getitem__                                                                                      │
│                                                                                                  │
│    4099 │   │   if is_single_key:                                                                │
│    4100 │   │   │   if self.columns.nlevels > 1:                                                 │
│    4101 │   │   │   │   return self._getitem_multilevel(key)                                     │
│ ❱  4102 │   │   │   indexer = self.columns.get_loc(key)                                          │
│    4103 │   │   │   if is_integer(indexer):                                                      │
│    4104 │   │   │   │   indexer = [indexer]                                                      │
│    4105 │   │   else:                                                                            │
│                                                                                                  │
│ d:\Program Files\Python\Python Venv\lc\lib\site-packages\pandas\core\indexes\base.py:3812 in     │
│ get_loc                                                                                          │
│                                                                                                  │
│   3809 │   │   │   │   and any(isinstance(x, slice) for x in casted_key)                         │
│   3810 │   │   │   ):                                                                            │
│   3811 │   │   │   │   raise InvalidIndexError(key)                                              │
│ ❱ 3812 │   │   │   raise KeyError(key) from err                                                  │
│   3813 │   │   except TypeError:                                                                 │
│   3814 │   │   │   # If we have a listlike key, _check_indexing_error will raise                 │
│   3815 │   │   │   #  InvalidIndexError. Otherwise we fall through and re-raise           

> The best prediction of my model based on RandomForestClassifier is at probability threshold **0.53**, when losses are the lowest ones: **1110.6**. Precision metric is the highest one at this threshold as well: **0.408**, although AUC metric is slightly higher than by chance: **0.52** and predict correct 122 defaults and mistakenly 2322 defaults as non-defaults. 

## Challenger model

I will use CatBoost as challenger model

In [77]:
scores = model.score(X_test, y_test)
scores

The best probability threshold based on min loss: 61


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ d:\Program Files\Python\Python Venv\lc\lib\site-packages\pandas\core\indexes\base.py:3805 in     │
│ get_loc                                                                                          │
│                                                                                                  │
│   3802 │   │   """                                                                               │
│   3803 │   │   casted_key = self._maybe_cast_indexer(key)                                        │
│   3804 │   │   try:                                                                              │
│ ❱ 3805 │   │   │   return self._engine.get_loc(casted_key)                                       │
│   3806 │   │   except KeyError as err:                                                           │
│   3807 │   │   │   if isinstance(casted_key, slice) or (                                         │
│   3808 │   │   │   │   isinstance(casted_key, abc.Iterable)                                      │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:167                                                    │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:196                                                    │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:7081                                        │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:7089                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'prob_thresh_%'

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 print(f"The best probability threshold based on min loss: {eval_metr[eval_metr.loss==eva     │
│   2 eval_metr[eval_metr.loss==eval_metr.loss.min()]                                              │
│   3                                                                                              │
│                                                                                                  │
│ d:\Program Files\Python\Python Venv\lc\lib\site-packages\pandas\core\frame.py:4102 in            │
│ __getitem__                                                                                      │
│                                                                                                  │
│    4099 │   │   if is_single_key:                                                                │
│    4100 │   │   │   if self.columns.nlevels > 1:                                                 │
│    4101 │   │   │   │   return self._getitem_multilevel(key)                                     │
│ ❱  4102 │   │   │   indexer = self.columns.get_loc(key)                                          │
│    4103 │   │   │   if is_integer(indexer):                                                      │
│    4104 │   │   │   │   indexer = [indexer]                                                      │
│    4105 │   │   else:                                                                            │
│                                                                                                  │
│ d:\Program Files\Python\Python Venv\lc\lib\site-packages\pandas

In [100]:
eval_metr = evaluate_metrics(model, X_test, y_test, params)

eval_metr 

[11/07/24 16:22:27] WARNING  d:\Program Files\Python\Python                                         ]8;id=266030;file://C:\Users\chemerys\AppData\Roaming\uv\python\cpython-3.10.15-windows-x86_64-none\lib\warnings.py\warnings.py]8;;\:]8;id=234750;file://C:\Users\chemerys\AppData\Roaming\uv\python\cpython-3.10.15-windows-x86_64-none\lib\warnings.py#109\109]8;;\
                             Venv\lc\lib\site-packages\sklearn\metrics\_classification.py:1531:                    
                             UndefinedMetricWarning: Precision is ill-defined and being set to 0.0                 
                             due to no predicted samples. Use `zero_division` parameter to control                 
                             this behavior.                                                                        
                               _warn_prf(average, modifier, f"{metric.capitalize()} is",                           
                             len(result))                                                                          
                                                                                                                   

                    WARNING  d:\Program Files\Python\Python                                         ]8;id=56645;file://C:\Users\chemerys\AppData\Roaming\uv\python\cpython-3.10.15-windows-x86_64-none\lib\warnings.py\warnings.py]8;;\:]8;id=179817;file://C:\Users\chemerys\AppData\Roaming\uv\python\cpython-3.10.15-windows-x86_64-none\lib\warnings.py#109\109]8;;\
                             Venv\lc\lib\site-packages\sklearn\metrics\_classification.py:1531:                    
                             UndefinedMetricWarning: Precision is ill-defined and being set to 0.0                 
                             due to no predicted samples. Use `zero_division` parameter to control                 
                             this behavior.                                                                        
                               _warn_prf(average, modifier, f"{metric.capitalize()} is",                           
                             len(result))                                                                          
                                                                                                                   

                    WARNING  d:\Program Files\Python\Python                                         ]8;id=371116;file://C:\Users\chemerys\AppData\Roaming\uv\python\cpython-3.10.15-windows-x86_64-none\lib\warnings.py\warnings.py]8;;\:]8;id=937666;file://C:\Users\chemerys\AppData\Roaming\uv\python\cpython-3.10.15-windows-x86_64-none\lib\warnings.py#109\109]8;;\
                             Venv\lc\lib\site-packages\sklearn\metrics\_classification.py:1531:                    
                             UndefinedMetricWarning: Precision is ill-defined and being set to 0.0                 
                             due to no predicted samples. Use `zero_division` parameter to control                 
                             this behavior.                                                                        
                               _warn_prf(average, modifier, f"{metric.capitalize()} is",                           
                             len(result))                                                                          
                                                                                                                   

[11/07/24 16:22:28] WARNING  d:\Program Files\Python\Python                                         ]8;id=522868;file://C:\Users\chemerys\AppData\Roaming\uv\python\cpython-3.10.15-windows-x86_64-none\lib\warnings.py\warnings.py]8;;\:]8;id=504237;file://C:\Users\chemerys\AppData\Roaming\uv\python\cpython-3.10.15-windows-x86_64-none\lib\warnings.py#109\109]8;;\
                             Venv\lc\lib\site-packages\sklearn\metrics\_classification.py:1531:                    
                             UndefinedMetricWarning: Precision is ill-defined and being set to 0.0                 
                             due to no predicted samples. Use `zero_division` parameter to control                 
                             this behavior.                                                                        
                               _warn_prf(average, modifier, f"{metric.capitalize()} is",                           
                             len(result))                                                                          
                                                                                                                   

                    WARNING  d:\Program Files\Python\Python                                         ]8;id=100728;file://C:\Users\chemerys\AppData\Roaming\uv\python\cpython-3.10.15-windows-x86_64-none\lib\warnings.py\warnings.py]8;;\:]8;id=21840;file://C:\Users\chemerys\AppData\Roaming\uv\python\cpython-3.10.15-windows-x86_64-none\lib\warnings.py#109\109]8;;\
                             Venv\lc\lib\site-packages\sklearn\metrics\_classification.py:1531:                    
                             UndefinedMetricWarning: Precision is ill-defined and being set to 0.0                 
                             due to no predicted samples. Use `zero_division` parameter to control                 
                             this behavior.                                                                        
                               _warn_prf(average, modifier, f"{metric.capitalize()} is",                           
                             len(result))                                                                          
                                                                                                                   

                    WARNING  d:\Program Files\Python\Python                                         ]8;id=851196;file://C:\Users\chemerys\AppData\Roaming\uv\python\cpython-3.10.15-windows-x86_64-none\lib\warnings.py\warnings.py]8;;\:]8;id=196319;file://C:\Users\chemerys\AppData\Roaming\uv\python\cpython-3.10.15-windows-x86_64-none\lib\warnings.py#109\109]8;;\
                             Venv\lc\lib\site-packages\sklearn\metrics\_classification.py:1531:                    
                             UndefinedMetricWarning: Precision is ill-defined and being set to 0.0                 
                             due to no predicted samples. Use `zero_division` parameter to control                 
                             this behavior.                                                                        
                               _warn_prf(average, modifier, f"{metric.capitalize()} is",                           
                             len(result))                                                                          
                                                                                                                   

                    WARNING  d:\Program Files\Python\Python                                         ]8;id=283753;file://C:\Users\chemerys\AppData\Roaming\uv\python\cpython-3.10.15-windows-x86_64-none\lib\warnings.py\warnings.py]8;;\:]8;id=467263;file://C:\Users\chemerys\AppData\Roaming\uv\python\cpython-3.10.15-windows-x86_64-none\lib\warnings.py#109\109]8;;\
                             Venv\lc\lib\site-packages\sklearn\metrics\_classification.py:1531:                    
                             UndefinedMetricWarning: Precision is ill-defined and being set to 0.0                 
                             due to no predicted samples. Use `zero_division` parameter to control                 
                             this behavior.                                                                        
                               _warn_prf(average, modifier, f"{metric.capitalize()} is",                           
                             len(result))                                                                          
                                                                                                                   

                    WARNING  d:\Program Files\Python\Python                                         ]8;id=997814;file://C:\Users\chemerys\AppData\Roaming\uv\python\cpython-3.10.15-windows-x86_64-none\lib\warnings.py\warnings.py]8;;\:]8;id=826612;file://C:\Users\chemerys\AppData\Roaming\uv\python\cpython-3.10.15-windows-x86_64-none\lib\warnings.py#109\109]8;;\
                             Venv\lc\lib\site-packages\sklearn\metrics\_classification.py:1531:                    
                             UndefinedMetricWarning: Precision is ill-defined and being set to 0.0                 
                             due to no predicted samples. Use `zero_division` parameter to control                 
                             this behavior.                                                                        
                               _warn_prf(average, modifier, f"{metric.capitalize()} is",                           
                             len(result))                                                                          
                                                                                                                   

The best probability threshold for catboost model based on min loss: 60


,prob_thresh_%,accuracy,precision,recall,f1,roc_auc,tn,fp,fn,tp,loss
catboost,10,0.29450,0.139983,0.927987,0.243269,0.567149,3622,13934,176,2268,3267.763360
catboost,15,0.49850,0.164574,0.761457,0.270652,0.611675,8109,9447,583,1861,2429.178905
catboost,20,0.64930,0.187671,0.561784,0.281352,0.611634,11613,5943,1071,1373,1852.733169
catboost,25,0.73835,0.204367,0.394435,0.269236,0.590331,13803,3753,1480,964,1540.385168
catboost,30,0.80040,0.223966,0.256956,0.239329,0.566505,15380,2176,1816,628,1334.584232
catboost,35,0.83985,0.242012,0.145663,0.181865,0.541076,16441,1115,2088,356,1217.294688
catboost,40,0.85765,0.238651,0.075286,0.114463,0.520925,16969,587,2260,184,1175.813012
catboost,45,0.86355,0.203742,0.040098,0.067009,0.509141,17173,383,2346,98,1168.793934
catboost,50,0.87750,0.285714,0.001637,0.003255,0.500534,17546,10,2440,4,1126.812200
catboost,55,0.87775,0.000000,0.000000,0.000000,0.499972,17555,1,2444,0,1126.597420


In [24]:
%reload_kedro
params['model_options'] = catalog.load("params:candidate_model.model_options")

# Make a model an fit it
model = model_pipeline(params['model_options'], params)
model.fit(X_train, y_train)
model


[11/07/24 22:33:19] INFO     Resolved project path as:                                              ]8;id=48058;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=944956;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#178\178]8;;\
                             /Users/andrejcemeris/Projects/Python3/lending-club.                                   
                             To set a different path, run '%reload_kedro <project_root>'                           

[11/07/24 22:33:21] INFO     Kedro project lending_club                                             ]8;id=911167;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=873405;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#144\144]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=561829;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=289273;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#145\145]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=604366;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=546451;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/ipython/__init__.py#151\151]8;;\

                    INFO     Loading data from params:candidate_model.model_options             ]8;id=722354;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=319393;file:///Users/andrejcemeris/anaconda3/envs/lc/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

0:	learn: 0.7238578	test: 0.7277791	best: 0.7277791 (0)	total: 58.4ms	remaining: 29.2s
100:	learn: 0.9679889	test: 0.9695254	best: 0.9695254 (100)	total: 6.41s	remaining: 25.3s
200:	learn: 0.9872888	test: 0.9870054	best: 0.9870054 (200)	total: 10.4s	remaining: 15.5s
300:	learn: 0.9913214	test: 0.9906550	best: 0.9906550 (300)	total: 13.8s	remaining: 9.13s
400:	learn: 0.9931161	test: 0.9915071	best: 0.9916715 (397)	total: 17.7s	remaining: 4.37s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.9916715249
bestIteration = 397

Shrink model to first 398 iterations.


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['home_ownership',
                                                   'sub_grade',
                                                   'verification_status']),
                                                 ('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['mths_since_rcnt_il']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('si...
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['open_rv_24m', 'open_acc_6m',
                                                   'max_bal_bc',
                                                   'mths_since_recent_inq',
                                                   'percent_bc_gt_75',
                                                   'bc_open_to_buy',
                                                   'tot_cur_bal',
                                                   'mo_sin_rcnt_rev_tl_op',
                                                   'acc_open_past_24mths'])])),
                ('smote', SMOTE(random_state=42)),
                ('catboostclassifier',
                 <catboost.core.CatBoostClassifier object at 0x11a08ac80>)])

In [25]:

eval_metr = evaluate_metrics(model, X_test, y_test, params)

eval_metr

,prob_thresh_%,accuracy,precision,recall,f1,roc_auc,tn,fp,fn,tp,loss
catboost,30,0.84245,0.320467,0.258183,0.285973,0.590985,16218,1338,1813,631,1141.553344
catboost,31,0.84615,0.323480,0.237316,0.273779,0.584111,16343,1213,1864,580,1136.470817
catboost,32,0.84980,0.328431,0.219313,0.263003,0.578442,16460,1096,1908,536,1129.991758
catboost,33,0.85225,0.329325,0.201718,0.250190,0.572265,16552,1004,1951,493,1128.769216
catboost,34,0.85465,0.330403,0.184534,0.236808,0.566236,16642,914,1993,451,1127.543194
catboost,35,0.85810,0.340872,0.172668,0.229223,0.563094,16740,816,2022,422,1118.496288
catboost,36,0.86065,0.346464,0.158347,0.217355,0.558383,16826,730,2057,387,1114.958953
catboost,37,0.86265,0.349553,0.144026,0.203999,0.553359,16901,655,2092,352,1113.937268
catboost,38,0.86395,0.351554,0.134206,0.194255,0.549873,16951,605,2116,328,1113.563392
catboost,39,0.86550,0.355634,0.123977,0.183859,0.546353,17007,549,2141,303,1112.278214


In [20]:
cb = CatBoostClassifier(**params['model_options']['regressor_options'], cat_features=(params['category']+[params['emp_len']]))